In [1]:
# https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/
from keras.preprocessing.text import Tokenizer

D:\Users\snandan.ORADEV\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# source text
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

In [3]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [4]:
encoded

[2,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 2,
 14,
 15,
 1,
 16,
 17,
 18,
 1,
 3,
 19,
 20,
 21]

In [6]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 22


In [22]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
    sequence = encoded[i-1:i+1]
    #print(sequence)
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 24


In [23]:
sequences

[[2, 1],
 [1, 3],
 [3, 4],
 [4, 5],
 [5, 6],
 [6, 7],
 [7, 8],
 [8, 9],
 [9, 10],
 [10, 11],
 [11, 12],
 [12, 13],
 [13, 2],
 [2, 14],
 [14, 15],
 [15, 1],
 [1, 16],
 [16, 17],
 [17, 18],
 [18, 1],
 [1, 3],
 [3, 19],
 [19, 20],
 [20, 21]]

In [33]:
import numpy as np
sequences = np.asarray(sequences)
# split into X and y elements
#from array import array
#sequences = array("i",sequences)
X, y = sequences[:,0],sequences[:,1]

In [34]:
X

array([ 2,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2, 14, 15,  1,
       16, 17, 18,  1,  3, 19, 20])

In [35]:
y

array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2, 14, 15,  1, 16,
       17, 18,  1,  3, 19, 20, 21])

In [37]:
from keras.utils import to_categorical
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)

In [38]:
y

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,

In [44]:
# Recurrent Neural Network
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Embedding

In [45]:
# define model
model = Sequential()
# The model uses a learned word embedding in the input layer. 
# This has one real-valued vector for each word in the vocabulary, 
# where each word vector has a specified length. In this case we will use a 10-dimensional projection. 
# The input sequence contains a single word, therefore the input_length=1.
model.add(Embedding(vocab_size, 10, input_length=1))
# The model has a single hidden LSTM layer with 50 units
model.add(LSTM(50))
# The output layer is comprised of one neuron for each word in the vocabulary and uses a softmax activation 
# function to ensure the output is normalized to look like a probability.
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 10)             220       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
 - 1s - loss: 3.0914 - acc: 0.0833
Epoch 2/500
 - 0s - loss: 3.0906 - acc: 0.2083
Epoch 3/500
 - 0s - loss: 3.0898 - acc: 0.2083
Epoch 4/500
 - 0s - loss: 3.0890 - acc: 0.2083
Epoch 5/500
 - 0s - loss: 3.0882 - acc: 0.2083
Epoch 6/500
 - 0s - loss: 3.0874 - acc: 0.1250
Epoch 7/500
 - 0s - loss: 3.0865 - acc: 0.1250
Epoch 8/500
 - 0s - loss: 3.0857 - acc: 0.1250
Epoch 9/500
 - 0s - loss: 3.0849 - acc: 0.1250
Epoch 10/500
 - 0s - loss: 3.0840 - acc: 0.1250
Epoch 11/500
 - 0s - loss: 3.0832 - acc: 0.1250
Epoch 12/500
 - 0s - loss: 3.0824 - acc: 0.1250
Epoch 13/500
 - 0s - loss: 3.0815 - acc: 0.1250
Epoch 14/500
 - 0s - loss: 3.0806 - acc: 0.1250
Epoch 15/500
 - 0s - loss: 3.0797 - acc: 0.1250
Epoch 16/500
 - 0s - loss: 3.0788 - acc: 0.1250
Epoch 17/500
 - 0s - loss: 3.0779 - acc: 0.1250
Epoch 18/500
 - 0s - loss: 3.0770 - acc: 0.1250
Epoch 19/500
 - 0s - loss: 3.0760 - acc: 0.1250
Epoch 20/500
 - 0s - loss: 3.0750 - acc: 0.1250
Epoch 21/500
 - 0s - loss: 3.0741 - acc: 0.1250
E

Epoch 171/500
 - 0s - loss: 1.9053 - acc: 0.5417
Epoch 172/500
 - 0s - loss: 1.8897 - acc: 0.5833
Epoch 173/500
 - 0s - loss: 1.8740 - acc: 0.5833
Epoch 174/500
 - 0s - loss: 1.8584 - acc: 0.5833
Epoch 175/500
 - 0s - loss: 1.8427 - acc: 0.5833
Epoch 176/500
 - 0s - loss: 1.8271 - acc: 0.6250
Epoch 177/500
 - 0s - loss: 1.8115 - acc: 0.6250
Epoch 178/500
 - 0s - loss: 1.7959 - acc: 0.6250
Epoch 179/500
 - 0s - loss: 1.7803 - acc: 0.6250
Epoch 180/500
 - 0s - loss: 1.7648 - acc: 0.6250
Epoch 181/500
 - 0s - loss: 1.7492 - acc: 0.6250
Epoch 182/500
 - 0s - loss: 1.7337 - acc: 0.6250
Epoch 183/500
 - 0s - loss: 1.7182 - acc: 0.6250
Epoch 184/500
 - 0s - loss: 1.7027 - acc: 0.6250
Epoch 185/500
 - 0s - loss: 1.6872 - acc: 0.6250
Epoch 186/500
 - 0s - loss: 1.6718 - acc: 0.6667
Epoch 187/500
 - 0s - loss: 1.6564 - acc: 0.6667
Epoch 188/500
 - 0s - loss: 1.6411 - acc: 0.6667
Epoch 189/500
 - 0s - loss: 1.6258 - acc: 0.6667
Epoch 190/500
 - 0s - loss: 1.6105 - acc: 0.7083
Epoch 191/500
 - 0s 

Epoch 339/500
 - 0s - loss: 0.3431 - acc: 0.8750
Epoch 340/500
 - 0s - loss: 0.3409 - acc: 0.8750
Epoch 341/500
 - 0s - loss: 0.3388 - acc: 0.8750
Epoch 342/500
 - 0s - loss: 0.3367 - acc: 0.8750
Epoch 343/500
 - 0s - loss: 0.3347 - acc: 0.8750
Epoch 344/500
 - 0s - loss: 0.3326 - acc: 0.8750
Epoch 345/500
 - 0s - loss: 0.3307 - acc: 0.8750
Epoch 346/500
 - 0s - loss: 0.3287 - acc: 0.8750
Epoch 347/500
 - 0s - loss: 0.3268 - acc: 0.8750
Epoch 348/500
 - 0s - loss: 0.3250 - acc: 0.8750
Epoch 349/500
 - 0s - loss: 0.3231 - acc: 0.8750
Epoch 350/500
 - 0s - loss: 0.3214 - acc: 0.8750
Epoch 351/500
 - 0s - loss: 0.3196 - acc: 0.8750
Epoch 352/500
 - 0s - loss: 0.3179 - acc: 0.8750
Epoch 353/500
 - 0s - loss: 0.3162 - acc: 0.8750
Epoch 354/500
 - 0s - loss: 0.3145 - acc: 0.8750
Epoch 355/500
 - 0s - loss: 0.3129 - acc: 0.8750
Epoch 356/500
 - 0s - loss: 0.3113 - acc: 0.8750
Epoch 357/500
 - 0s - loss: 0.3097 - acc: 0.8750
Epoch 358/500
 - 0s - loss: 0.3082 - acc: 0.8750
Epoch 359/500
 - 0s 

In [48]:
# evaluate
in_text = 'Jack'
print(in_text)
encoded = tokenizer.texts_to_sequences([in_text])[0]
encoded = np.asarray(encoded)
yhat = model.predict_classes(encoded, verbose=0)
for word, index in tokenizer.word_index.items():
	if index == yhat:
		print(word)

Jack
and


In [49]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
	in_text, result = seed_text, seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = array(encoded)
		# predict a word in the vocabulary
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + ' ' + out_word
	return result

# source text
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
	sequence = encoded[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# split into X and y elements
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)
# evaluate
print(generate_seq(model, tokenizer, 'Jack', 6))

Vocabulary Size: 22
Total Sequences: 24
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 10)             220       
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_2 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
 - 1s - loss: 3.0918 - acc: 0.0417
Epoch 2/500
 - 0s - loss: 3.0910 - acc: 0.0417
Epoch 3/500
 - 0s - loss: 3.0903 - acc: 0.1667
Epoch 4/500
 - 0s - loss: 3.0895 - acc: 0.1667
Epoch 5/500
 - 0s - loss: 3.0887 - acc: 0.1667
Epoch 6/500
 - 0s - loss: 3.0879 - acc: 0.2083
Epoch 7/500
 - 0s - loss: 3.0871 - acc: 0.2083
Epoch 8/500
 - 0s - loss: 3.

Epoch 155/500
 - 0s - loss: 2.1474 - acc: 0.3333
Epoch 156/500
 - 0s - loss: 2.1344 - acc: 0.3333
Epoch 157/500
 - 0s - loss: 2.1214 - acc: 0.3333
Epoch 158/500
 - 0s - loss: 2.1083 - acc: 0.3333
Epoch 159/500
 - 0s - loss: 2.0953 - acc: 0.3750
Epoch 160/500
 - 0s - loss: 2.0823 - acc: 0.3750
Epoch 161/500
 - 0s - loss: 2.0693 - acc: 0.3750
Epoch 162/500
 - 0s - loss: 2.0564 - acc: 0.3750
Epoch 163/500
 - 0s - loss: 2.0434 - acc: 0.4167
Epoch 164/500
 - 0s - loss: 2.0305 - acc: 0.4167
Epoch 165/500
 - 0s - loss: 2.0176 - acc: 0.4167
Epoch 166/500
 - 0s - loss: 2.0047 - acc: 0.4167
Epoch 167/500
 - 0s - loss: 1.9918 - acc: 0.4167
Epoch 168/500
 - 0s - loss: 1.9790 - acc: 0.4583
Epoch 169/500
 - 0s - loss: 1.9662 - acc: 0.5000
Epoch 170/500
 - 0s - loss: 1.9534 - acc: 0.5417
Epoch 171/500
 - 0s - loss: 1.9406 - acc: 0.5417
Epoch 172/500
 - 0s - loss: 1.9279 - acc: 0.5417
Epoch 173/500
 - 0s - loss: 1.9151 - acc: 0.5417
Epoch 174/500
 - 0s - loss: 1.9024 - acc: 0.5417
Epoch 175/500
 - 0s 

Epoch 323/500
 - 0s - loss: 0.5300 - acc: 0.8750
Epoch 324/500
 - 0s - loss: 0.5255 - acc: 0.8750
Epoch 325/500
 - 0s - loss: 0.5212 - acc: 0.8750
Epoch 326/500
 - 0s - loss: 0.5168 - acc: 0.8750
Epoch 327/500
 - 0s - loss: 0.5126 - acc: 0.8750
Epoch 328/500
 - 0s - loss: 0.5084 - acc: 0.8750
Epoch 329/500
 - 0s - loss: 0.5042 - acc: 0.8750
Epoch 330/500
 - 0s - loss: 0.5001 - acc: 0.8750
Epoch 331/500
 - 0s - loss: 0.4961 - acc: 0.8750
Epoch 332/500
 - 0s - loss: 0.4921 - acc: 0.8750
Epoch 333/500
 - 0s - loss: 0.4881 - acc: 0.8750
Epoch 334/500
 - 0s - loss: 0.4843 - acc: 0.8750
Epoch 335/500
 - 0s - loss: 0.4804 - acc: 0.8750
Epoch 336/500
 - 0s - loss: 0.4767 - acc: 0.8750
Epoch 337/500
 - 0s - loss: 0.4729 - acc: 0.8750
Epoch 338/500
 - 0s - loss: 0.4693 - acc: 0.8750
Epoch 339/500
 - 0s - loss: 0.4657 - acc: 0.8750
Epoch 340/500
 - 0s - loss: 0.4621 - acc: 0.8750
Epoch 341/500
 - 0s - loss: 0.4586 - acc: 0.8750
Epoch 342/500
 - 0s - loss: 0.4551 - acc: 0.8750
Epoch 343/500
 - 0s 

Epoch 491/500
 - 0s - loss: 0.2450 - acc: 0.8750
Epoch 492/500
 - 0s - loss: 0.2445 - acc: 0.8750
Epoch 493/500
 - 0s - loss: 0.2441 - acc: 0.8750
Epoch 494/500
 - 0s - loss: 0.2437 - acc: 0.8750
Epoch 495/500
 - 0s - loss: 0.2433 - acc: 0.8750
Epoch 496/500
 - 0s - loss: 0.2429 - acc: 0.8750
Epoch 497/500
 - 0s - loss: 0.2425 - acc: 0.8750
Epoch 498/500
 - 0s - loss: 0.2421 - acc: 0.8750
Epoch 499/500
 - 0s - loss: 0.2417 - acc: 0.8750
Epoch 500/500
 - 0s - loss: 0.2413 - acc: 0.8750
Jack and jill came tumbling after hill
